In [55]:
import ee
import geemap
import numpy as np

In [56]:
## authenticates the session with EE
ee.Authenticate()

# creates the connection with EE. 
ee.Initialize()

In [57]:
# Define a region of interest (ROI) for the analysis
roi = ee.Geometry.Rectangle([162.277817, -77.740157, 163.272100, -77.576571])  
# Coordinate range that covers the Taylor Valley lakes


In [71]:
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1') \
            .filterBounds(roi)\
            .filterDate('2020-01-01', '2020-12-31')

In [91]:
# preprocessing for cloud masking
def maskL8sr(image):
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    qa = image.select('BQA')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
            .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [92]:
landsat8 = landsat_collection.map(maskL8sr)

In [93]:
# define ROI's in specific images that will define lake ice/soil

roi_ice = ee.Geometry.Rectangle([163.077952, -77.616808, 163.077080, -77.616903])
roi_soil = ee.Geometry.Rectangle([163.078070, -77.625204, 163.075507, -77.625340])

In [94]:
# Define endmembers (example: using pseudo-invariant features)
# can add more endmembers in later, but need to select image and crop down to 
# specific regions in a single image
lake1 = ee.Image('LANDSAT/LC08/C02/T2/LC08_055116_20231205')
soil1 = ee.Image('LANDSAT/LC08/C02/T2/LC08_055116_20231205')

# clip original image down to the specific endmember spots
lakeice = lake1.clip(roi_ice)
soil = soil1.clip(roi_soil)

ice_mean = lakeice.reduceRegion(ee.Reducer.mean()).values()
soil_mean = soil.reduceRegion(ee.Reducer.mean()).values()

#endmember3 = ee.Image('YOUR_ENDMEMBER_IMAGE3')  # Replace with your endmember image

In [95]:
# Create an image with endmembers as bands
endmembers = ee.Image.cat([ice_mean, soil_mean])

In [102]:
# Function to apply SMA to an image
def apply_sma(image):
    fractional_abundances = image.unmix(endmembers, True, True)
    return fractional_abundances

In [103]:
# Apply SMA to the image collection
fractional_abundances_col = landsat8.map(apply_sma)

In [104]:
# Export results (example: export first image)
first_image = fractional_abundances_col.first()
task = ee.batch.Export.image.toDrive(
    image=first_image,
    description='fractional_abundances',
    folder = "SMA_EarthEngine",
    scale=30,
    region=roi
)
task.start()
print('Export task status:', task.status())

Export task status: {'state': 'READY', 'description': 'fractional_abundances', 'priority': 100, 'creation_timestamp_ms': 1722379801148, 'update_timestamp_ms': 1722379801148, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'FHMIQPBOUH43CHIVTNCR2JCV', 'name': 'projects/earthengine-legacy/operations/FHMIQPBOUH43CHIVTNCR2JCV'}


In [106]:
print(first_image.getInfo())

EEException: Image.addBands, argument 'dstImg': Invalid type.
Expected type: Image<unknown bands>.
Actual type: List<Float>.
Actual value: [14611.983050847457, 17625.79661016949, 16727.389830508473, 14660.338983050848, 14422.915254237287, 14164.983050847455, 11888.271186440677, 5208.237288135593, 5177.237288135592, 24608.864406779656, 5038.220338983051, 30048.0, 0.0, 7321.0, 6345.0, 14333.610169491523, 393.38983050847463]

In [101]:
# allows to check for errors in google drive upload that don't show up as an
# outright error in the above code chunk. 
# Monitor tasks
def monitor_tasks():
    tasks = ee.batch.Task.list()
    for task in tasks:
        print(f'Task ID: {task.id}, Status: {task.status()}')

# Print status of all tasks
monitor_tasks()

print("Export task started. Check Google Drive for results.")

Task ID: 7EQL7YVLPFIPMWRPMNZMSKJ5, Status: {'state': 'FAILED', 'description': 'fractional_abundances', 'priority': 100, 'creation_timestamp_ms': 1722379640390, 'update_timestamp_ms': 1722379647506, 'start_timestamp_ms': 1722379646349, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Image.addBands, argument 'dstImg': Invalid type.\nExpected type: Image<unknown bands>.\nActual type: List<Float>.\nActual value: [14611.983050847457, 17625.79661016949, 16727.389830508473, 14660.338983050848, 14422.915254237287, 14164.983050847455, 11888.271186440677, 5208.237288135593, 5177.237288135592, 24608.864406779656, 5038.220338983051, 30048.0, 0.0, 7321.0, 6345.0, 14333.610169491523, 393.38983050847463]", 'id': '7EQL7YVLPFIPMWRPMNZMSKJ5', 'name': 'projects/earthengine-legacy/operations/7EQL7YVLPFIPMWRPMNZMSKJ5'}
Task ID: H3IV246AX5RBFAZ2KTSQDFD4, Status: {'state': 'FAILED', 'description': 'fractional_abundances', 'priority': 100, 'creation_timestamp_ms': 1722379602928, 'update_timestamp

Task ID: F3HTQ2FACBIDJYN5NHANW2PB, Status: {'state': 'FAILED', 'description': 'SMA of Taylor Valley with dummy endmembers', 'priority': 100, 'creation_timestamp_ms': 1722361911697, 'update_timestamp_ms': 1722361918899, 'start_timestamp_ms': 1722361918106, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "ImageCollection.reduce: Error in map(ID=LC08_053116_20170104):\nImage.select: Can't add a band named 'B4' to image because a band with this name already exists. Existing bands: [B4, B3].", 'id': 'F3HTQ2FACBIDJYN5NHANW2PB', 'name': 'projects/earthengine-legacy/operations/F3HTQ2FACBIDJYN5NHANW2PB'}
Task ID: DJ2SUSRUF427LFPEQ5TVO24U, Status: {'state': 'FAILED', 'description': 'SMA of Taylor Valley with dummy endmembers', 'priority': 100, 'creation_timestamp_ms': 1722361714706, 'update_timestamp_ms': 1722361721670, 'start_timestamp_ms': 1722361721144, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "ImageCollection.reduce: Error in map(ID=LC08_053116_20170104):\nImag

Task ID: O7DT6BLGJWS3Q37KLP5JHT2Q, Status: {'state': 'COMPLETED', 'description': 'SMA_Export', 'priority': 100, 'creation_timestamp_ms': 1722355190447, 'update_timestamp_ms': 1722355253329, 'start_timestamp_ms': 1722355195222, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1DwEgKluWuHjA6k3et3m8tNQbYVkcEPQv'], 'attempt': 1, 'batch_eecu_usage_seconds': 5.252835750579834, 'id': 'O7DT6BLGJWS3Q37KLP5JHT2Q', 'name': 'projects/earthengine-legacy/operations/O7DT6BLGJWS3Q37KLP5JHT2Q'}
Task ID: 7GWDCOXIXMUJJLA5BIN4PTZ6, Status: {'state': 'FAILED', 'description': 'SMA_Export', 'priority': 100, 'creation_timestamp_ms': 1722355166534, 'update_timestamp_ms': 1722355176304, 'start_timestamp_ms': 1722355175677, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Image.unmix, argument 'endmembers': Invalid type.\nExpected type: List<List<Float>>.\nActual type: Image<[array]>.", 'id': '7GWDCOXIXMUJJLA5BIN4PTZ6', 'name': 'projects/earthengine-legacy/operatio